# Wrangle and Analyze Data
## Table of Contents
<ul>
<li><a href="#1.-Gathering">Gathering</a></li>
<li><a href="#2.-Assessing">Assessing</a></li>
<li><a href="#3.-Cleaning">Cleaning</a></li>
<li><a href="#4.-Analysis">Analysis</a></li>
</ul>

# 1. Gathering

The file `twitter-archive-enhanced.csv` has been provided. We will need to download the file image-predictions.tsv and generate a tweet-json.txt connecting to the Twitter API.

In [ ]:
import pandas as pd
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image

In [ ]:
# Use the request library to download image_predictions.tsv
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url)

with open('./image-predictions.tsv', 'wb') as f:
    f.write(r.content)

# Retrieve HTTP meta-data
print(r.status_code)
print(r.headers['content-type'])
print(r.encoding)

In [ ]:
# Get favorite and retweet count from Twitter API
#import tweepy
#from tweepy import OAuthHandler
#import json
#from timeit import default_timer as timer

#consumer_key = '-'
#consumer_secret = '-'
#access_token = '-'
#access_secret = '-'

#auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
#auth.set_access_token(access_token, access_secret)
#api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

# Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive

# Vars
#df_1 = pd.read_csv('twitter-archive-enhanced.csv')
#tweet_ids = df_1.tweet_id.values
#fails_dict = {}
#count = 0

#start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
#with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
#    for tweet_id in tweet_ids:
#        count += 1
#        print(str(count) + ": " + str(tweet_id))
#        try:
#            tweet = api.get_status(tweet_id, tweet_mode='extended')
#            print("Success")
#            json.dump(tweet._json, outfile)
#            outfile.write('\n')
#        except tweepy.TweepError as e:
#            print("Fail")
#            fails_dict[tweet_id] = e
#            pass
#end = timer()
#print(end - start)
#print(fails_dict)

Once we have all the files downloaded, the next step will be assessing the data looking for Quality and Tidy issues. 

## 1.1 Create one dataframe from each file

In [ ]:
df_twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_image_predictions = pd.read_csv('image-predictions.tsv', sep='\t')
df_tweet_json = pd.read_json('tweet-json', lines=True)

# 2. Assessing

## 2.1 Assess Twitter Archive Enhanced Dataframe

In [ ]:
df_twitter_archive

In [ ]:
df_twitter_archive.info()

In [ ]:
df_twitter_archive.duplicated().sum()

In [ ]:
df_twitter_archive.describe()

In [ ]:
# Check for replies & retweets
print('Number of replies:', df_twitter_archive.in_reply_to_user_id.count())
print('Number of retweets:', df_twitter_archive.retweeted_status_id.count())

In [ ]:
df_twitter_archive.rating_denominator.value_counts()

In [ ]:
df_twitter_archive.rating_numerator.value_counts()

In [ ]:
df_twitter_archive.name.value_counts()

In [ ]:
df_twitter_archive.query('name == "a"')

In [ ]:
df_twitter_archive.source.value_counts()

## 2.2 Assess Image Predictions Dataframe

In [ ]:
df_image_predictions

In [ ]:
df_image_predictions.info()

In [ ]:
df_image_predictions.describe()

In [ ]:
# There seems to be some inconsistency with the capitalization of the p1, p2 and p3 columns
df_image_predictions.p1.value_counts()

In [ ]:
df_image_predictions.duplicated().sum()

In [ ]:
# Are there tweets with duplicated images?
df_image_predictions.jpg_url.duplicated().sum()

In [ ]:
df_image_predictions[df_image_predictions.jpg_url.duplicated()]

In [ ]:
# Check one of these tweets with duplicated image url
df_image_predictions.query('jpg_url == "https://pbs.twimg.com/media/CtVAvX-WIAAcGTf.jpg"')

In [ ]:
# Check both tweets in the archive dataframe
df_twitter_archive.query('tweet_id == 780601303617732608')

In [ ]:
df_twitter_archive.query('tweet_id == 821813639212650496')

Seems that the duplicated images corresponds with the retweets, that we should clean in the clean step.

In [ ]:
df_image_predictions.tweet_id.value_counts()

## 2.3 Assess Tweet Json Dataframe

In [ ]:
df_tweet_json

In [ ]:
df_tweet_json.info()

In [ ]:
df_tweet_json.describe()

In [ ]:
# As the max favorite_count diverges a lot from the mean, let´s check if it could be a error
df_tweet_json.favorite_count.sort_values(ascending=False)

# Looking at the results, it doesn´t seem so. There are at least 6 tweets with over 100k favorites

In [ ]:
# Let´s do the same with retweet count
df_tweet_json.retweet_count.sort_values(ascending=False)

# Again, these results seem normal, so no apparent input/import errors here

In [ ]:
# Check for null values on fav and retweet count
print('Null values in favorite_count?', df_tweet_json.favorite_count.isnull().any())
print('Null values in retweet_count?', df_tweet_json.retweet_count.isnull().any())
print('Null values in user?', df_tweet_json.user.isnull().any())

## 2.4 Issues 

### Quality issues assesed

1. `tweet_id` is an integer but it should be a string as it is not used to compute anything
2. `timestamp` column should have datetime format
3. There are replies and retweets in our data
4. There are tweets without images
5. `name` column contains strings that aren´t names as 'a', 'an', 'the'...
6. `source` column contains html code
7. `p1`, `p2`, `p3` columns have inconsistent capitalization
8. Not all dogs have been properly matched to a particular breed in columns
9. There are too many unneded columns

### Tidyness

1. One of the rules of Tiny Data is that each variable forms a column. In this dataframe we have one variable `dog_stages` split in 4 columns `doggo`, `floofer`, `pupper`, `puppo`.
2. Another rule about Tiny Data is that each type of obserbational unit forms a table. We are analyzing tweets, but we have the information split in three different dataframes. We should combine them.

# 3. Cleaning

In [ ]:
# Let´s creat copies of our dataframes
df2_twitter_archive = df_twitter_archive.copy()
df2_image_predictions = df_image_predictions.copy()
df2_tweet_json = df_tweet_json.copy()

## 3.1. Tidiness issues

Let´s with the tidyness issues detected in our assessment phase. To solve the detected problem we will do the next actions:

1. Create one dog_stage column from 4 different ones
2. Combine DataFrames

### 3.1.1. Create one dog_stage column from 4 different ones

**Code**

In [ ]:
# Loop over the dataframe & create the dog_stage column
dog_stage = []

for index, row in df2_twitter_archive.iterrows():
    if row['doggo'] == 'doggo': dog_stage.append('doggo')
    elif row['floofer'] == 'floofer': dog_stage.append('floofer')
    elif row['pupper'] == 'pupper': dog_stage.append('pupper')
    elif row['puppo'] == 'puppo': dog_stage.append('puppo')
    else: dog_stage.append('None')

df2_twitter_archive['dog_stage'] = dog_stage

In [ ]:
# Delete old columns
df2_twitter_archive.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis=1, inplace=True)

**Test** 

In [ ]:
df2_twitter_archive.dog_stage.value_counts()

### 3.1.2 Combine DataFrames

**Code**

In [ ]:
# First, the json dataframe contains a lot of columns we are not interested in. 
# Let´s create another one keeping only the columns we are interested in.
df2_tweet_json = df2_tweet_json[['id', 'favorite_count', 'retweet_count']]

# Rename id column to tweet_id to ease the later merge
df2_tweet_json.rename(columns={'id' : 'tweet_id'}, inplace=True)

df2_tweet_json.head()

In [ ]:
# Merge all dataframes so we have all the information related to a tweet in the same one 
aux = pd.merge(df2_twitter_archive, 
                 df2_tweet_json,
                 left_on='tweet_id',
                 right_on='tweet_id',
                 how='left')

df_merged_dataframes = pd.merge(aux,
                               df2_image_predictions,
                               left_on='tweet_id',
                               right_on='tweet_id',
                               how='left')

**Test**

In [ ]:
# Check that we have a single dataframe with all the columns from the 3 previous ones
df_merged_dataframes.info()

## 3.2 Quality Issues

Next, we´ll fix the quality issues raised during our assessment with the following actions:

1. Convert tweet_id from integer to string
2. Apply datetime format to timestamp column
3. Delete replies and retweets
4. Delete tweets without image
5. Fix wrong names
6. Strip html tags from source column
7. Fix inconsistent capitalization in `p1`, `p2`, `p3` colums
8. Create one `breed` column
9. Delete unneded columns

### 3.2.1 Convert tweet_id from integer to string

**Code**

In [ ]:
# Change column type
df_merged_dataframes.tweet_id = df_merged_dataframes.tweet_id.astype(str)

**Test**

In [ ]:
from pandas.api.types import is_string_dtype
is_string_dtype(df_merged_dataframes.tweet_id)

### 3.2.2 Apply datetime format to timestamp column

**Code**

In [ ]:
df_merged_dataframes.timestamp = pd.to_datetime(df_merged_dataframes.timestamp)

**Test**

In [ ]:
from pandas.api.types import is_datetime64_any_dtype
is_datetime64_any_dtype(df_merged_dataframes.timestamp)

### 3.2.3 Delete replies and retweets

**Code**

In [ ]:
# Delete tweets that are replies from the dataframe
df_merged_dataframes = df_merged_dataframes[df_merged_dataframes.in_reply_to_status_id.isnull()] 

# Delete tweets that are retweets from the dataframe
df_merged_dataframes = df_merged_dataframes[df_merged_dataframes.retweeted_status_id.isnull()] # 181 results

**Test**

In [ ]:
# Check for replies & retweets
print('Number of replies:', df_merged_dataframes.in_reply_to_user_id.count())
print('Number of retweets:', df_merged_dataframes.retweeted_status_id.count())

### 3.2.4 Delete tweets without image

**Code**

In [ ]:
df_merged_dataframes = df_merged_dataframes[df_merged_dataframes.jpg_url.notnull()]

**Test**

In [ ]:
df_merged_dataframes.jpg_url.isnull().any()

### 3.2.5 Fix wrong names

**Code** 

In [ ]:
# There are names that are not names as 'a', 'an', 'the' that seem clear input errors.
# Let´s clean them converting all lowercase word to 'None'
df_merged_dataframes.name = df_merged_dataframes.name.mask(df_merged_dataframes.name.str.match(r'^[a-z]+$'), 'None')

**Test**

In [ ]:
df_merged_dataframes.name.value_counts()

### 3.2.6 Strip html tags from source column

**Code**

In [ ]:
import re
df_merged_dataframes.source = df_merged_dataframes.source.apply(lambda x: re.sub('<[^<]+?>', '', x))

**Test**

In [ ]:
df_merged_dataframes.source.value_counts()

### 3.2.7 Fix inconsistent capitalization in p1, p2, p3 colums

**Code**

In [ ]:
# Convert to lowercase each value in those columns
for _ in ['p1', 'p2', 'p3']:
  df_merged_dataframes[_] = df_merged_dataframes[_].str.lower()

**Test**

In [ ]:
# Check visually that everything is lowercase
df_merged_dataframes.p1.value_counts()

### 3.2.8 Create one breed column 

**Code**

In [ ]:
# In our dataframe, our dogs are categorized in the p1, p2, p3 columns, but not all of them
# have been positively matched with a particular breed. In this cell we are going to create
# a new column breed with the breed when we have it validated and 'None' in any other case.
breed = []

for index, row in df_merged_dataframes.iterrows():
    if row['p1_dog'] == True:
        breed.append(row['p1'])
    elif row['p2_dog'] == True:
        breed.append(row['p2'])
    elif row['p3_dog'] == True:
        breed.append(row['p3'])
    else:
        breed.append('None')

df_merged_dataframes['breed'] = breed

In [ ]:
# Let´s drop all the dogs that haven´t been properly categorized
df_merged_dataframes = df_merged_dataframes.query('breed != "None"')

**Test**

In [ ]:
df_merged_dataframes.breed.value_counts()

### 3.2.9 Delete unneded columns

In [ ]:
# There are columns without data that we are not going to use
df_merged_dataframes.info()

**Code**

In [ ]:
# Drop unneded columns
df_merged_dataframes.drop(columns=['in_reply_to_status_id', 'in_reply_to_user_id', \
                                   'retweeted_status_id', 'retweeted_status_user_id',\
                                   'retweeted_status_timestamp'], inplace=True)

**Test**

In [ ]:
df_merged_dataframes.info()

## 4. Analysis

In [ ]:
# First, let's save the dataframe to a csv file
df_merged_dataframes.to_csv('twitter_archive_master.csv')

In this final phase of the project, we will answer some questions around the data we´ve been gathering, assessing and cleaning. 

**Questions**

1. Which are our most favorited dogs?
2. Which are our most retweeted dogs?
3. Is there any correlation between favorite_count and retweet_count?
4. Is there any correlation between the number of pictures and the favorites_count?
4. Which is the most favorited dog breed?
5. Which breeds receive more favorites in average?


### 4.1 Let´s see the most favorited dogs

No very useful, but after so many work with the data I´m curious about how look the most popular dogs.

In [ ]:
top5_favorited_dogs = df_merged_dataframes.sort_values(by='favorite_count', ascending=False).head(5).jpg_url

for dog_url in top5_favorited_dogs:
    response = requests.get(dog_url, stream=True)
    img = Image.open(response.raw)
    plt.imshow(img)
    plt.show()

### 4.2 Let´s see the most retweeted dogs

Again, not very useful. Our expectation is to see some of the most favorited dogs also in this short list. 

In [ ]:
top5_retweeted_dogs = df_merged_dataframes.sort_values(by='retweet_count', ascending=False).head(5).jpg_url

for dog_url in top5_retweeted_dogs:
    response = requests.get(dog_url, stream=True)
    img = Image.open(response.raw)
    plt.imshow(img)
    plt.show()

### 4.3 Is there any correlation between favorite_count and retweet_count?

As we´ve seen with our top 5, we expect that the correlation will be positive between both characteristics. 

In [ ]:
# Variables
chart_title="Correlation favorite_count / retweet_count"
chart_xlabel="favorite count"
chart_ylabel="retweet count"

# Plot
ax = plt.subplots(figsize=(16,9))
ax = sns.regplot(x=df_merged_dataframes['favorite_count'], y=df_merged_dataframes['retweet_count'])

# Function that we will use during the exercise to decorate our charts
# This function was creatd in the first module of the Data Analyst Nanodegree
def decorate(title, xlabel, ylabel, xticks=None, xticklabels=None, yticks=None, yticklabels=None):
    ax.set_title(title, fontsize=22)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)  
    if xticks is not None:
        ax.set_xticks(xticks)
    if xticklabels is not None:
        ax.set_xticklabels(xticklabels, rotation=70)
    if yticks is not None:
        ax.set_xticks(yticks)
    if yticklabels is not None:
        ax.set_xticklabels(yticklabels, rotation=70)
    ax.grid(alpha=1)
    sns.set(style="dark")

# Decorate
decorate(chart_title, chart_xlabel, chart_ylabel)

plt.show;

As we were expecting, the correlation between favorite_count and retweet_count is positive.

### 4.4 Is there any correlation between the number of pictures and the favorites_count?

Maybe tweets with more than 1 picture receive more favorites than tweets with only one image. Let´s check it.

In [ ]:
# Variables
chart_title="Correlation number of pictures / favorite_count"
chart_xlabel="Number of pictures"
chart_ylabel="Favorites count"

# Plot
ax = plt.subplots(figsize=(16,9))
ax = sns.regplot(x=df_merged_dataframes['img_num'], y=df_merged_dataframes['favorite_count'])

# Decorate
decorate(chart_title, chart_xlabel, chart_ylabel)

plt.show;

We can appreciate a weak positive correlation between the number of pictures and the favorites count obtained. 

### 4.5 Which is the most favorited dog breed?

In [ ]:
# Create a group breed / favorite count
grp = pd.DataFrame(df_merged_dataframes.groupby(['breed']).sum().favorite_count)

# Create a dataframe with the top 10 breeds and their favorite_count
df_favorite_breed = pd.DataFrame(grp.sort_values('favorite_count', ascending=False)).head(10)

# Plot a graph
# Variables
chart_title="Most favorited breeds"
chart_xlabel="Breed"
chart_ylabel="Favorite Count"
chart_xticks=np.arange(len(df_favorite_breed))
chart_xticklabels=df_favorite_breed.index

# Plot a bar graph
fig, ax = plt.subplots(figsize=(16,9))
ax.bar(chart_xticks, df_favorite_breed.favorite_count)

# Decorate
decorate(chart_title, chart_xlabel, chart_ylabel, chart_xticks, chart_xticklabels)

plt.show;

### 4.6 Which breeds receive more favorites in average?

The previous graph shows the most favorited breeds without having into account the number of samples of each dog. That could lead us to wrong conclusions, so now we are going to compute the popularity of dog breeds according to the number of favorites and the number of samples of each one in our data.

In [ ]:
# Create a dataframe with the columns we need
df_aux = pd.DataFrame(df_merged_dataframes.groupby(['breed']).sum().favorite_count)
df_aux['breed_count'] = pd.DataFrame(df_merged_dataframes.groupby(['breed']).count().tweet_id)

# Calculate the ration favorites / breed 
df_aux['favs_per_breed'] = df_aux['favorite_count'] / df_aux['breed_count']

# Order the results
df_aux = pd.DataFrame(df_aux.sort_values('favs_per_breed', ascending=False)).head(10)

# Create a dataframe with the top 10 breeds and their favorite_count
df_aux = pd.DataFrame(df_aux.sort_values('favs_per_breed', ascending=False)).head(10)

# Plot a graph
# Variables
chart_title="Average favorites per breed"
chart_xlabel="Breed"
chart_ylabel="Favorite Count"
chart_xticks=np.arange(len(df_aux))
chart_xticklabels=df_aux.index

# Plot a bar graph
fig, ax = plt.subplots(figsize=(16,9))
ax.bar(chart_xticks, df_aux.favs_per_breed)

# Decorate
decorate(chart_title, chart_xlabel, chart_ylabel, chart_xticks, chart_xticklabels)

plt.show;